In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dense, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
import numpy as np


In [2]:
# list devices for tensorflow
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32  
DIRECTORY = "datasets"

In [4]:
train_dataset = image_dataset_from_directory(
    DIRECTORY,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

validation_dataset = image_dataset_from_directory(
    DIRECTORY,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

class_names = train_dataset.class_names
num_classes = len(class_names)


Found 1570 files belonging to 5 classes.
Using 1256 files for training.
Found 1570 files belonging to 5 classes.
Using 314 files for validation.


In [5]:
class_names

['Alternaria_Leaf_Spot',
 'Bacterial_Spot',
 'Downy_Mildew',
 'Healthy_Petchay',
 'Undefined']

In [6]:
base_model = MobileNetV3Large(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False  



12683000/12683000 [==============================] - 3s 0us/step


In [7]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 256, 256, 3)          0         ['input_1[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 128, 128, 16)         432       ['rescaling[0][0]']           
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 128, 128, 16)         64        ['Conv[0][0]']                
 lization)                                                                                    

In [27]:
print(len(model.layers))

266


In [28]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset)

Epoch 1/10
40/40 [==============================] - 369s 8s/step - loss: 0.2946 - accuracy: 0.9061 - val_loss: 0.2577 - val_accuracy: 0.9299
Epoch 2/10
40/40 [==============================] - 302s 8s/step - loss: 0.1997 - accuracy: 0.9530 - val_loss: 0.2659 - val_accuracy: 0.9204
Epoch 3/10
40/40 [==============================] - 301s 8s/step - loss: 0.1854 - accuracy: 0.9475 - val_loss: 0.2704 - val_accuracy: 0.9236
Epoch 4/10
40/40 [==============================] - 301s 7s/step - loss: 0.1608 - accuracy: 0.9570 - val_loss: 0.2663 - val_accuracy: 0.9268
Epoch 5/10
38/40 [===========================>..] - ETA: 13s - loss: 0.1323 - accuracy: 0.9696

In [12]:
base_model.trainable = True

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history_fine = model.fit(train_dataset,
                         epochs=1,
                         validation_data=validation_dataset)


40/40 [==============================] - 340s 7s/step - loss: 0.8163 - accuracy: 0.7014 - val_loss: 0.2446 - val_accuracy: 0.9395


In [13]:
img_path ="C:/Users/Owner/Desktop/petchay_doctor-main/notebook3/datasets/Bacterial_Spot/2300_jpg.rf.f67d2e323d9def14a94c2f0bb7f7cc5f.jpg"

img = tf.keras.utils.load_img(
    img_path, target_size= IMAGE_SIZE
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [19]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions)

print(
    "This image most likely belongs to {} ."
    .format(class_names[np.argmax(score)])
)

1/1 [==============================] - 6s 6s/step
This image most likely belongs to Bacterial_Spot .


In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 256, 256, 3)          0         ['input_1[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 128, 128, 16)         432       ['rescaling[0][0]']           
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 128, 128, 16)         64        ['Conv[0][0]']                
 lization)                                                                                    

In [26]:
model.save('/petchay_doctor-main/notebook3/')

TypeError: Binding inputs to tf.function failed due to `missing a required argument: 'inputs'`. Received args: (TensorSpec(shape=(None, None, None, 16), dtype=tf.float32, name=None),) and kwargs: {'training': True} for signature: (self, inputs, training=<captured_default_value>, mask=<captured_default_value>).

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('bokchoymodel.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:
class_labels = class_names  # Replace with your actual class labels

# Write the class labels to a labels.txt file
with open("petchay_labels.txt", "w") as f:
    for label in class_labels:
        f.write(label + "\n")